# Bayesian Network Probability Density Map Generation

Generate the PDM for a SAR scenario using the technique outlined by Lin and Goodrich[1] using Bayesian Networks

[1]: L. Lin and M. A. Goodrich, ‘A Bayesian approach to modeling lost person behaviors based on terrain features in Wilderness Search and Rescue’, Comput Math Organ Theory, vol. 16, no. 3, pp. 300–323, Jul. 2010, doi: [10.1007/s10588-010-9066-2](https://doi.org/10.1007/s10588-010-9066-2).

In [ ]:
import sys, os

sys.path.insert(0, os.path.join(os.getcwd(), "../src")) # run f

In [ ]:
import numpy as np
from scipy import signal as sps
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl

from loguru import logger
from typing import List, Tuple
from copy import copy
import pydantic
import enum

In [ ]:
# Install from package at https://github.com/iwishiwasaneagle/jsim/releases/latest or add use it from within the project
from jsim.Environment import Environment
from jsim.Simulation import Simulation
from jsim.Meta import Action, State

## Hexagonal Environment

Use the offset coord system https://www.redblobgames.com/grids/hexagons/#coordinates-offset, the odd-r variant in particular

In [ ]:
class OffsetCoord(pydantic.BaseModel):
    col: int
    row: int

class AxialCoord(pydantic.BaseModel):
    q: int
    r: int

class PixelCoord(pydantic.BaseModel):
    x: float
    y: float

class Direction(enum.IntEnum):
    br = 0
    tr = 1
    t = 2
    tl = 3
    bl = 4
    b = 5

In [ ]:

class HexOddQPDM(Environment):
    def __init__(self, m: int = 20, psim: Simulation = None) -> None:
        super().__init__(psim=psim)

        self._shape = (m,m)

        self.pdm = np.zeros(self._shape)

    def as_mpl_polygons(self,
                        cmap: mpl.colors.LinearSegmentedColormap = mpl.cm.get_cmap('Spectral'),
                        size: float = np.sqrt(1/3)
                        ) -> List[patches.RegularPolygon]:
        norm = mpl.colors.Normalize(vmin=np.min(self.pdm), vmax=np.max(self.pdm))

        offsets = []
        colors = []
        i,j = np.meshgrid(np.arange(self.shape[0]), np.arange(self.shape[1]))
        for col,row in zip(i.flatten(), j.flatten()):
            z = self.pdm[row,col]
            colors.append(cmap(norm(z)))
            offsets.append(OffsetCoord(col=col,row=row))
        return HexOddQPDM.offsets_to_mpl_polygons(offsets, colors, size)

    @staticmethod
    def offsets_to_mpl_polygons(offsets: List[OffsetCoord],
                                colors: List[Tuple[float]] = None,
                                size: float = np.sqrt(1/3)
                        ) -> List[patches.RegularPolygon]:
        arr = []
        if colors is None:
            colors = [None for _ in offsets]

        for o,c in zip(offsets,colors):
            arr.append(HexOddQPDM.offset_to_mpl_polygon(o,c, size))
        return arr

    @staticmethod
    def offset_to_mpl_polygon(offset: OffsetCoord, color: Tuple[float, float, float, float] = None, size: float = np.sqrt(1/3)) -> patches.RegularPolygon:
        pixel = HexOddQPDM.offset_to_pixel(offset, size)
        return patches.RegularPolygon((pixel.x, pixel.y), numVertices=6, radius=size, orientation=np.pi/6, fc=color, ec='k')

    @property
    def shape(self):
        return self._shape

    @staticmethod
    def offset_to_axial(offset: OffsetCoord) -> AxialCoord:
        q = offset.col
        r = offset.row - (offset.col - (offset.col&1)) / 2 # use bitewise and to detect even, as it also works with negative numbers
        return AxialCoord(q=q,r=r)

    @staticmethod
    def axial_to_offset(axial: AxialCoord) -> OffsetCoord:
        col = axial.q
        row = axial.r + (axial.q - (axial.q&1))/2
        return OffsetCoord(col=col, row=row)

    @staticmethod
    def axial_to_pixel(axial: AxialCoord, size: float) -> PixelCoord:
        x = size * ((3./2) * axial.q)
        y = size * (np.sqrt(3)/2 * axial.q + np.sqrt(3)* axial.r)
        return PixelCoord(x=x,y=y)

    @staticmethod
    def offset_to_pixel(offset: OffsetCoord, size: float = np.sqrt(1/3)) -> PixelCoord:
        # https://www.redblobgames.com/grids/hexagons/#hex-to-pixel-offset
        axial = HexOddQPDM.offset_to_axial(offset)
        pixel = HexOddQPDM.axial_to_pixel(axial, size)
        return pixel

    @staticmethod
    def neighbor_coord(offset: OffsetCoord, direction: Direction) -> OffsetCoord:
        direction_differences = [
            # for explanation: https://www.redblobgames.com/grids/hexagons/#neighbors-offset
            # even cols
            [[+1,  0], [+1, -1], [ 0, -1],
             [-1, -1], [-1,  0], [ 0, +1]],
            # odd cols
            [[+1, +1], [+1,  0], [ 0, -1],
             [-1,  0], [-1, +1], [ 0, +1]],
            ]
        parity = offset.col & 1
        diff = direction_differences[parity][direction]
        return OffsetCoord(col=offset.col + diff[0], row=offset.row + diff[1])

    @staticmethod
    def neighbors_coord(offset: OffsetCoord) -> List[OffsetCoord]:
        return [HexOddQPDM.neighbor_coord(offset, d) for d in Direction]

    @staticmethod
    def distance_between_a_b( a: OffsetCoord, b: OffsetCoord) -> float:
        # https://www.redblobgames.com/grids/hexagons/#distances-offset
        def axial_distance_between_a_b(c: AxialCoord, d: AxialCoord) -> float:
            return (abs(c.q - d.q) + abs(c.q + c.r - d.q - d.r) + abs(c.r - d.r)) / 2
        ac = HexOddQPDM.offset_to_axial(a)
        bd = HexOddQPDM.offset_to_axial(b)
        return  axial_distance_between_a_b(ac, bd)

    def reset(self) -> State:
        pass

    def step(self, pa: Action) -> Tuple[State, float]:
        pass

def test_neighbor_coord():
    # even col, any row
    directions = [d for d in Direction]
    expected_neighbors = [(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(0,1)]
    expected_neighbors = [OffsetCoord(col=f[0], row=f[1]) for f in expected_neighbors]

    for d, e in zip(directions, expected_neighbors):
        assert HexOddQPDM.neighbor_coord(
                        OffsetCoord(col=0,row=0), d
                        ) == e

    # odd col, any row
    expected_neighbors = [(2,1),(2,0),(1,-1),(0,0),(0,1),(1,1)]
    expected_neighbors = [OffsetCoord(col=f[0], row=f[1]) for f in expected_neighbors]

    for d, e in zip(directions, expected_neighbors):
        assert HexOddQPDM.neighbor_coord(
                        OffsetCoord(col=1,row=0), d
                    ) == e
test_neighbor_coord()

In [ ]:
pdm = HexOddQPDM(m=9)

polygons = pdm.as_mpl_polygons()

fig, ax = plt.subplots(1)
ax.set_aspect('equal')

for p in polygons:
    ax.add_patch(p)

pos = OffsetCoord(col=2,row=3)
for p in HexOddQPDM.offsets_to_mpl_polygons(HexOddQPDM.neighbors_coord(pos)):
    ax.add_patch(p)

ax.add_patch(HexOddQPDM.offset_to_mpl_polygon(pos,(1.,1.,1.,1.)))
plt.autoscale(enable=True)
plt.show()